In [1]:
import pandas as pd
from tree_sitter import Language, Parser

import os
os.chdir('/home/dev/function_parser/')

from function_parser.language_data import LANGUAGE_METADATA
from process import DataProcessor

In [11]:
! rm -rf mkdir /home/dev/.vendor 
! mkdir /home/dev/.vendor
! rm -rf mkdir /home/dev/.build
! mkdir /home/dev/.build

! git clone https://github.com/tree-sitter/tree-sitter-python /home/dev/.vendor/tree-sitter-python

Cloning into '/home/dev/.vendor/tree-sitter-python'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 1903 (delta 55), reused 50 (delta 30), pack-reused 1820
Receiving objects: 100% (1903/1903), 17.25 MiB | 15.42 MiB/s, done.
Resolving deltas: 100% (1211/1211), done.


In [12]:
Language.build_library(
  # Store the library in the `build` directory
  '/home/dev/.build/py-tree-sitter-languages.so',

  # Include one or more languages
  [
    #'vendor/tree-sitter-go',
    #'vendor/tree-sitter-javascript',
    '/home/dev/.vendor/tree-sitter-python'
  ]
)

True

In [15]:
language_name = 'python'

language = Language('/home/dev/.build/py-tree-sitter-languages.so', language_name)
parser = Parser()
parser.set_language(language)

In [77]:
blob = bytes("""
def foo():
    if bar:
        baz()
    if bar2:
        baz2()
""", "utf8")
blob[0:15]


b'\ndef foo():\n   '

In [49]:
tree = parser.parse(blob)

In [99]:
def print_ast(cursor, blob, depth = 0):
    depth+=1
    # print(dir(cursor.node))
    if cursor.node.type == 'identifier':
        name = blob[cursor.node.start_byte:cursor.node.end_byte].decode("utf8")
        print(' '*depth + name)
    else:
        print(' '*depth + cursor.node.type) 
    
    if cursor.goto_first_child():
        while cursor:
            print_tree(cursor, blob, depth)
            if not cursor.goto_next_sibling():
                break
        cursor.goto_parent()



In [100]:
cursor = tree.walk()
print_ast(cursor, blob)


module
  function_definition
   def
   foo
   parameters
    (
    )
   :
   block
    if_statement
     if
     bar
     :
     block
      expression_statement
       call
        baz
        argument_list
         (
         )
    if_statement
     if
     bar2
     :
     block
      expression_statement
       call
        baz2
        argument_list
         (
         )


In [ ]:
def ast2graph(cursor, blob, depth = 0):
    depth+=1
    # print(dir(cursor.node))
    if cursor.node.type == 'identifier':
        name = blob[cursor.node.start_byte:cursor.node.end_byte].decode("utf8")
        print(' '*depth + name)
    else:
        print(' '*depth + cursor.node.type) 
    
    if cursor.goto_first_child():
        while cursor:
            print_tree(cursor, blob, depth)
            if not cursor.goto_next_sibling():
                break
        cursor.goto_parent()